In [1]:
import pandas as pd
from functools import reduce  
from tqdm import tqdm

In [36]:
##load anno for polyfun

bl='/gpfs/commons/groups/knowles_lab/data/ldsc/polyfun/baselineLF2.2.UKB/baselineLF2.2.UKB.'
deepsea='/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/annotations/annotations_high_h2/deepsea/deepsea_high_h2_chr'
roadmap='/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/annotations/annotations_high_h2/roadmap/roadmap_high_h2_chr'
glass_lab_enformer='/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/annotations/annotations_high_h2/glass_lab_enformer/glass_lab_enformer_high_h2_chr'
glass_lab_enformer_abs='/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/annotations/annotations_high_h2/glass_lab_enformer_abs/glass_lab_enformer_abs_high_h2_chr'
enformer='/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/annotations/annotations_high_h2/enformer/enformer_high_h2_chr'
glass_lab='/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/annotations/annotations_high_h2/glass_lab/glass_lab_high_h2_chr'


sumstat = '/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/summary_stats/alzheimers/fixed_alzheimers/processed/sbayesrc/bellenguez_hg19_chr'

suffix= '.annot.gz'
output_path='/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/annotations/4SBayesRC/chr'

In [15]:
test['ID'] = test.apply(lambda row: "%s_%s_%s_%s" % (row['CHR'], row['BP'], row['A1'], row['A2']), axis=1)
test

,CHR,SNP,BP,A1,A2,deepsea-zscore_2-1168,deepsea-zscore_3-1168,deepsea-zscore_1-1932,deepsea-zscore_2-1938,deepsea-zscore_3-1938,deepsea-zscore_2-1939,deepsea-zscore_3-1939,ID
0,22,rs587755077,16050115,G,A,0,0,0,0,0,0,0,22_16050115_G_A
1,22,rs587654921,16050213,C,T,1,0,0,1,0,1,0,22_16050213_C_T
2,22,rs587769434,16050527,C,A,1,0,1,1,1,1,0,22_16050527_C_A
3,22,rs2186465,16050678,C,T,0,0,0,0,0,0,0,22_16050678_C_T
4,22,22:16050739_TA_T,16050739,TA,T,0,0,0,0,0,0,0,22_16050739_TA_T
...,...,...,...,...,...,...,...,...,...,...,...,...,...
271694,22,rs561893765,51239794,C,A,0,0,0,0,0,0,0,22_51239794_C_A
271695,22,rs529322970,51240084,G,C,0,0,0,0,0,0,0,22_51240084_G_C
271696,22,rs202228854,51240820,C,T,0,0,0,0,0,0,0,22_51240820_C_T
271697,22,rs568168135,51241386,C,G,0,0,0,0,0,0,0,22_51241386_C_G


In [30]:
def merge_annotation(chrom):
    annotations = [bl, deepsea, roadmap, glass_lab_enformer, glass_lab_enformer_abs, enformer, glass_lab]
    annotation_chr=[]
    for i in range(0, len(annotations)):
        if annotations[i] == '/gpfs/commons/groups/knowles_lab/data/ldsc/polyfun/baselineLF2.2.UKB/baselineLF2.2.UKB.':
            annotation_name = annotations[i] + str(chrom) + '.annot.parquet'
            anno = pd.read_parquet(annotation_name, engine='pyarrow')
        else:
            annotation_name = annotations[i] + str(chrom) + suffix
            anno = pd.read_csv(annotation_name, compression='gzip', sep = '\t')
        
        anno['ID'] = anno.apply(lambda row: "%s_%s_%s_%s" % (row['CHR'], row['SNP'], row['A1'], row['A2']), axis=1)
        anno = anno.drop(anno.columns[[0,1,2,3,4]], axis = 1)
        #anno = anno.drop(anno.columns[[0,2,3]], axis = 1)
        annotation_chr.append(anno)
            
    anno_merge = reduce(lambda left, right:pd.merge(left,right,on=["ID"]),annotation_chr)
    anno_merge.insert(1, 'Intercept', 1)
    print(anno_merge.shape)
    return(anno_merge)
   # anno_merge.to_csv(output_path+str(chrom)+".tsv",index = False,sep='\t')
   
        
    
    




In [31]:
chr21 = merge_annotation(21)

(272383, 327)


In [21]:
chr21.ID

0          21_9411245_C_A
1          21_9411410_C_T
2          21_9411485_C_A
3          21_9411500_G_T
4          21_9411602_T_C
               ...       
272378    21_48118443_T_G
272379    21_48119252_G_A
272380    21_48119259_C_A
272381    21_48119510_C_T
272382    21_48119634_G_A
Name: ID, Length: 272383, dtype: object

In [22]:
cojo = pd.read_csv(sumstat+'21.cojo', sep = '\t')

In [25]:
cojo

,SNP,A1,A2,freq,b,se,p,N
0,rs560822176,T,C,0.0023,-0.0308,0.2369,0.8966,42140
1,rs148979894,T,C,0.0129,0.0389,0.0628,0.5355,81962
2,rs200796725,C,T,0.9870,-0.0328,0.0659,0.6190,81962
3,rs112055405,G,A,0.9915,-0.0411,0.0610,0.5007,471522
4,rs186331370,A,G,0.0104,0.0454,0.0618,0.4624,81962
...,...,...,...,...,...,...,...,...
208377,rs3827268,G,A,0.9166,-0.0054,0.0147,0.7153,487511
208378,rs406179,G,A,0.6392,-0.0064,0.0084,0.4491,487511
208379,rs9980376,A,G,0.0609,-0.0233,0.0173,0.1774,487511
208380,rs9983726,C,T,0.9396,0.0217,0.0173,0.2113,487511


In [28]:
cojo['ID'] =  cojo.apply(lambda row: "%s_%s_%s_%s" % (21, row['SNP'], row['A1'], row['A2']), axis=1)


In [34]:
check = pd.merge(cojo, chr21, on='ID', how='inner')
check

,SNP_x,A1,A2,freq,b,se,p,N,ID,SNP_x,...,glass_lab-microglia_astrocyte-enhancer-none,glass_lab-microglia_astrocyte-enhancer_promoter_union_atac_500-none,glass_lab-microglia_neuron-atac-none,glass_lab-microglia_neuron-enhancer-none,glass_lab-microglia_neuron-enhancer_promoter_union_atac_500-none,glass_lab-microglia_neuron_astrocyte_oligodendrocyte-atac-none,glass_lab-microglia_neuron_astrocyte_oligodendrocyte-enhancer_promoter_union_atac_500-none,glass_lab-microglia_oligodendrocyte-atac-none,glass_lab-microglia_oligodendrocyte-enhancer_promoter_union_atac_500-none,glass_lab-neuron-enhancer_promoter_union_atac_500-none
0,rs112055405,G,A,0.9915,-0.0411,0.0610,0.5007,471522,21_rs112055405_G_A,rs112055405,...,0,0,0,0,0,0,0,0,0,0
1,rs149462070,A,C,0.0169,0.0002,0.0583,0.9977,455165,21_rs149462070_A_C,rs149462070,...,0,0,0,0,0,0,0,0,0,0
2,rs578031753,C,A,0.9995,-0.6942,0.4458,0.1194,42140,21_rs578031753_C_A,rs578031753,...,0,0,0,0,0,0,0,0,0,0
3,rs117486011,C,T,0.9884,0.0194,0.0468,0.6784,469677,21_rs117486011_C_T,rs117486011,...,0,0,0,0,0,0,0,0,0,0
4,rs200234628,G,T,0.9885,0.0038,0.0457,0.9332,469677,21_rs200234628_G_T,rs200234628,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153797,rs3827268,G,A,0.9166,-0.0054,0.0147,0.7153,487511,21_rs3827268_G_A,rs3827268,...,0,0,0,0,0,0,0,0,0,0
153798,rs406179,G,A,0.6392,-0.0064,0.0084,0.4491,487511,21_rs406179_G_A,rs406179,...,0,0,0,0,0,0,0,0,0,0
153799,rs9980376,A,G,0.0609,-0.0233,0.0173,0.1774,487511,21_rs9980376_A_G,rs9980376,...,0,0,0,0,0,0,0,0,0,0
153800,rs9983726,C,T,0.9396,0.0217,0.0173,0.2113,487511,21_rs9983726_C_T,rs9983726,...,0,0,0,0,0,0,0,0,0,0


In [35]:
check[1:10]

,SNP_x,A1,A2,freq,b,se,p,N,ID,SNP_x,...,glass_lab-microglia_astrocyte-enhancer-none,glass_lab-microglia_astrocyte-enhancer_promoter_union_atac_500-none,glass_lab-microglia_neuron-atac-none,glass_lab-microglia_neuron-enhancer-none,glass_lab-microglia_neuron-enhancer_promoter_union_atac_500-none,glass_lab-microglia_neuron_astrocyte_oligodendrocyte-atac-none,glass_lab-microglia_neuron_astrocyte_oligodendrocyte-enhancer_promoter_union_atac_500-none,glass_lab-microglia_oligodendrocyte-atac-none,glass_lab-microglia_oligodendrocyte-enhancer_promoter_union_atac_500-none,glass_lab-neuron-enhancer_promoter_union_atac_500-none
1,rs149462070,A,C,0.0169,0.0002,0.0583,0.99770,455165,21_rs149462070_A_C,rs149462070,...,0,0,0,0,0,0,0,0,0,0
2,rs578031753,C,A,0.9995,-0.6942,0.4458,0.11940,42140,21_rs578031753_C_A,rs578031753,...,0,0,0,0,0,0,0,0,0,0
3,rs117486011,C,T,0.9884,0.0194,0.0468,0.67840,469677,21_rs117486011_C_T,rs117486011,...,0,0,0,0,0,0,0,0,0,0
4,rs200234628,G,T,0.9885,0.0038,0.0457,0.93320,469677,21_rs200234628_G_T,rs200234628,...,0,0,0,0,0,0,0,0,0,0
5,rs559494670,T,C,0.0027,-0.1613,0.1496,0.28100,440038,21_rs559494670_T_C,rs559494670,...,0,0,0,0,0,0,0,0,0,0
6,rs374689396,T,G,0.0032,-0.0030,0.1209,0.98010,462058,21_rs374689396_T_G,rs374689396,...,0,0,0,0,0,0,0,0,0,0
7,rs369629676,A,G,0.0012,-0.3541,0.1827,0.05259,443137,21_rs369629676_A_G,rs369629676,...,0,0,0,0,0,0,0,0,0,0
8,rs371247752,G,A,0.9900,-0.0190,0.0548,0.72930,471522,21_rs371247752_G_A,rs371247752,...,0,0,0,0,0,0,0,0,0,0
9,rs558207057,G,T,0.9960,-0.0032,0.0785,0.96780,469677,21_rs558207057_G_T,rs558207057,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
for i in tqdm(range(1 ,9)):
    print('running on chr%s'%i)
    merge_annotation(i)
    